In [1]:
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
'''
Dopo il caricamento del file, si procede a creare un file apposito in cui vi sono sono i titoli
(già processati) con le rispettive label (create mediante coni rispettivi tag)
'''

data = pd.read_csv('DB/Preprocessed_data.csv')

data = data[data['tags'].notna()]
data = data[data['processed_title'].notna()]



tmp_tags = []
tmp_titles = []
 
for tags in data["tags"]:
    
    tmp_tags.append(tags)

for titles in data["processed_title"]:
    
    tmp_titles.append(titles)

#print(tmp_tags)
#print(tmp_titles)
       
i = 0
tmp_label = []
final = []
while(i < len(data)):
    
    #Controllo se la domanda ha un solo tag
    if(len(tmp_tags[i].split('|')) == 1):
        
            tmp_label = "__label__" + tmp_tags[i].split('|')[0] + " " + tmp_titles[i]
            
    #Controllo se la domanda ha almeno due tag
    if(len(tmp_tags[i].split('|')) > 1):
        
        j = 0
        
        while(j < len(tmp_tags[i].split('|'))):
            
            #Controllo se è la prima-esima iterazione (analisi del primo tag)
            if ( j == 0):
                tmp_label = "__label__" + tmp_tags[i].split('|')[j] + " " + tmp_titles[i]
                j = j+1
                
            #Controllo se è la seconda-esami iterazione (analisi dal secondo tag)
            else:
                tmp_label = " __label__" + tmp_tags[i].split('|')[j] + " " + tmp_label
                j = j+1
                
    final.append(tmp_label)
    i = i+1

#print("------------")
#print(final)
#Creo df per il training formato da tags + processed title 
#__label__ processed_title
df = pd.DataFrame(final)

train, valid = train_test_split(df, test_size=0.2)

train.to_csv('DB/fasttext.train')
valid.to_csv('DB/fasttext.valid')


In [3]:

'''
Qui si procede all'allenamento del modello.

Come primo step, dopo aver controllato la grandezza del file, bisogna dividere in due il file con le label ('GFG.csv'):

	>> wc GFG.csv
		15404  169582 1401900 GFG.csv    #CONTIENE 15404 ESEMPI

	>> head -n 12404 GFG.csv > stack.train
    >> tail -n 3000 GFG.csv > stack.valid

Successivamente, si può passare all'addastramento del modello.

L'utilizzo del parametro "lr" permette di modificare la velocità di apprendimento del modello.
L'aumento (o la diminuzione) della volecità di apprendimento dell'algoritmo permette di far si che il modello
cambi dopo l'elaborazione di ciascun esempio.
Con un tasso di apprendimento pari a 0, si ha che il modello non cambia e quindi non apprende nulla, mentre
si hanno dei buoni valori di tasso di apprendimento nell'intervallo 0.1 - 1.0

L'utilizzo del parametro "epoch" permette di avere un maggiore tasso di apprendimento.
Dato che, in base ad un impostazione predefinita di fastText, esso vede ogni esempio di allenamento
solo 5 volte durante l'addestramento (numero molto piccolo dato che non abbiamo solo ... esempi di addestramento),
si può aumentare il numero di volte in cui viene visualizzato ogni esempio (noto anche numero di epoche)
utilizzando il paramento "epoch".

L'utilizzo del parametro "wordNgrams" permette di migliorare le prestazioni del modello. 
Piuttosto che utilizzare solo gli unigrammi,si vanno ad utilizzare i bigrammi di parole,
poichè l'ordine delle parole è importante.
Nel fastText, con "unigramma" ci si riferisce ad una singola parola.
Nel fastText, con "bigrammi" ci si riferisce a due parole consecutive.
'''

model = fasttext.train_supervised(input='DB/fasttext.train', lr=0.8, epoch=40, wordNgrams=2)







In [4]:
model.test('DB/fasttext.valid')

(18207, 0.817048387982644, 0.29705065995726754)

In [5]:
model.save_model("DB/model_stack.bin")

In [6]:
model.predict("os using python installed default install reason asking use yum install going install mysqlpython client version python installed os centos installed manually using altinstall option python mysqlpython likley installed default version python machine go see mysqlpython need either install using easyinstall pip download mysqlpython package manually install add ever installed python path search mysqlpython machine see installed give better understanding going",k=-1,threshold=0.01)

(('__label__pip',
  '__label__django',
  '__label__installation',
  '__label__virtualenv',
  '__label__linux',
  '__label__macos',
  '__label__ubuntu',
  '__label__pandas',
  '__label__centos'),
 array([0.14572966, 0.12606479, 0.02434609, 0.02118622, 0.01620425,
        0.01179107, 0.01125851, 0.01040747, 0.0103855 ]))

In [8]:
model.predict("problem loading compressed gz csv file url ", k=-1, threshold= 0.01)

(('__label__csv',
  '__label__url',
  '__label__encoding',
  '__label__pandas',
  '__label__utf-8'),
 array([0.78492862, 0.02154011, 0.01472156, 0.01181447, 0.01141658]))